In [1]:
from csv import reader
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from xgboost import Booster


features = ['년','월', '시', 
           '기온(°C)','강수량(mm)', '풍속(m/s)', 
           '습도(%)', '적설(cm)','요일_0', '요일_1', 
           '요일_2', '요일_3','요일_4', '요일_5', '요일_6',]

out = pd.DataFrame(pd.read_csv("data/api/out.csv"))
out['date'] = pd.to_datetime(out['date'], errors='coerce')
out['년'] = out['date'].dt.year - 2010
out['월'] = out['date'].dt.month
out['시'] = out['date'].dt.hour
out['요일'] = out['date'].dt.weekday
#out['대여장소'] = out['number']
#out['반납장소'] = out['number']
out['기온(°C)'] = out['temperature']
out['강수량(mm)'] = out['rain']
out['풍속(m/s)'] = out['wind']
out['습도(%)'] = out['humidity']
out['적설(cm)'] = out['snow']
#out = out.join(pd.get_dummies(out['요일'], prefix="요일"))

out['요일_0'] = 0
out['요일_1'] = 0
out['요일_2'] = 0
out['요일_3'] = 0
out['요일_4'] = 0
out['요일_5'] = 0
out['요일_6'] = 0
for i in range(out.shape[0]):    
    if out.loc[i, '요일'] == 0:
        out.loc[i, '요일_0'] = 1
    if out.loc[i, '요일'] == 1:
        out.loc[i, '요일_1'] = 1
    if out.loc[i, '요일'] == 2:
        out.loc[i, '요일_2'] = 1
    if out.loc[i, '요일'] == 3:
        out.loc[i, '요일_3'] = 1
    if out.loc[i, '요일'] == 4:
        out.loc[i, '요일_4'] = 1
    if out.loc[i, '요일'] == 5:
        out.loc[i, '요일_5'] = 1
    if out.loc[i, '요일'] == 6:
        out.loc[i, '요일_6'] = 1

#대여소번호 지정
location = out['number'].values        
        
out = out.drop(['number','date', 'temperature', 'rain', 'wind', 'humidity', 'snow', '요일'], 1)

print(out)
# 30개 
rental = np.array([221, 342, 568, 216, 316,
                   210, 340, 326, 106, 259,
                   302, 639, 540, 341, 212,
                   222, 336, 116, 124, 385,
                   247, 162, 329, 118, 128,
                   346, 907, 144, 529, 501])

# 10개. 고빈도
rental = np.array([207, 502, 113, 2102, 1210, 
                   152, 1308, 2219, 1906, 907])

# 저장된 모델 로드해서 테스트


loaded_model = xgb.Booster()
loaded_modelr = xgb.Booster()
# 수요106 모델 로드
loaded_model.load_model("data/model/수요" + str(int(location)) + ".model")
loaded_modelr.load_model("data/model/공급" + str(int(location)) + ".model")

dtest = xgb.DMatrix(out)

#수요
predictions = loaded_model.predict(dtest)
#공급
predictionsr = loaded_modelr.predict(dtest)

prediction = predictionsr-predictions
print(prediction)

   년  월  시  기온(°C)  강수량(mm)  풍속(m/s)  습도(%)  적설(cm)  요일_0  요일_1  요일_2  요일_3  \
0  7  3  5     1.9        0      0.6     75       0     0     0     0     0   

   요일_4  요일_5  요일_6  
0     1     0     0  
[0.03754175]


In [ ]:
#MAE 출력

import xgboost as xgb
location = 106
ts106 = ts[ts['대여장소'] == location]
x_trains, x_tests, y_trains, y_tests = train_test_split(ts106[features], ts106['빈도수'], test_size=0.2, random_state=15)
tr106 = tr[tr['반납장소'] == location]
x_trainr, x_testr, y_trainr, y_testr = train_test_split(tr106[features], tr106['빈도수'], test_size=0.2, random_state=15)

loaded_model = xgb.Booster()
loaded_modelr = xgb.Booster()

loaded_model.load_model("data/model/수요" + str(int(location)) + ".model")
loaded_modelr.load_model("data/model/공급" + str(int(location)) + ".model")

dtests = xgb.DMatrix(x_tests)
dtestr = xgb.DMatrix(x_testr)

predictions = loaded_model.predict(dtests)
predictionsr = loaded_modelr.predict(dtestr)

act = y_testr-y_tests
dif = predictionsr-predictions

mae = mean_absolute_error(act, dif)
print(mae)